<div align="center">

# CMSC 320 Final Project: How Much Does Electricity Cost the US?

##### A Tutorial and Analysis by Jay Desmarais
***

</div>

<div align="center">

### Introduction

</div>

With a rising concern for the world's health and climate change, all things energy are a big topic. Researchers are finding the best ways to harness renewable energy efficiently, companies are turning away from the use of fossil fuels, people are buying electric cars, and electric people are starting to rely on their electric companies a little less, by installing their own solar panels. With all this happening, there are still large providers in every state that a huge portion of the United States rely on to get their electricity, and over the years, their means of obtaining and distributing said electricity may vary, whether they start relying more on nuclear plants, solar panels, or other more renewable sources, but how has this, among many other obstables, investments, failures, or more impacted the price of electricity. 

This analysis and tutorial will take a look at the U.S Department of Energy's data on the average price of electricity in each US state from the years 1990 - 2020 in their dataset titled "Average Price (Cents/kilowatthour) by State by Provider, 1990-2020". By taking a look at this data, we hope to find possible trends in prices accross the nation, in specific states or regions, and across time.

<div align="center">

### Required Tools and Libraries

</div>

The following tools and libraries are required for different aspects of this tutorial:

1. urllib.request 
2. pandas
3. numpys
4. matplotlib.pyplot
5. re
6. datetime
7. bs4


To import these packages, the following code can be used: 

In [35]:
import urllib.request as request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
from datetime import datetime
from bs4 import BeautifulSoup

<div align="center">

### Data Collection

</div>

In [39]:

url = "https://www.energy.gov/sites/default/files/2022-05/PDL-05-04-2022.json"
response = request.urlopen(url)
data = json.loads(response.read())

for dataset in data['dataset']:
    if dataset['title'] == 'Annual Electricity Price by State':
        energy_dataset = dataset
        break

data_url = energy_dataset['distribution'][0]['downloadURL']

df = pd.read_excel(data_url, header=1)

df.head()

,Year,State,Industry Sector Category,Residential,Commercial,Industrial,Transportation,Other,Total
0,2020,AK,Total Electric Industry,22.57,19.58,15.88,0.00,NaN,19.82
1,2020,AL,Total Electric Industry,12.58,11.55,5.87,0.00,NaN,9.84
2,2020,AR,Total Electric Industry,10.41,8.61,5.89,13.32,NaN,8.32
3,2020,AZ,Total Electric Industry,12.27,10.11,6.07,9.38,NaN,10.44
4,2020,CA,Total Electric Industry,20.45,17.53,14.27,10.07,NaN,18.00


<div align="center">

### Data Management and Representation

</div>

<div align="center">

### Exploratory Data Analysis

</div>

<div align="center">

### Hypothesis Testing

</div>

<div align="center">

### Communication of Insights Attained

</div>